朴素贝叶斯是生成学习方法。    生成方法由训练数据学习联合概率分布P(X,Y),然后求得后验概率分布P(Y|X)。利用训练数据学习P(X|Y)和P(Y)的估计，得到联合概率分布： P(X,Y) = P(Y)(X|Y)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
import math

In [7]:
#data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, :])
    #print(data)
    return data[:,:-1], data[:,-1]

In [8]:
a = create_data()
print(a)

(array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 

In [9]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [10]:
X_test[0],y_test[0]

(array([4.8, 3. , 1.4, 0.1]), 0.0)

## GaussianNB 高斯朴素贝叶斯 ##

In [11]:
class NaiveBayes:
    def __init__(self):
        self.model = None
        
    #数学期望
    @staticmethod
    def mean(X):
        return sum(X) / float(len(X))
    
    #标准差(方差)
    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x-avg, 2) for x in X]) / float(len(X)))
    
    #概率密度函数
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent
    
    #处理X_train
    def summarize(self, train_data):
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
        return summaries
    
    #分类别求出数学期望和标准差
    def fit(self, X, y):
        labels = list(set(y))
        data = {label:[] for label in labels}
        for f, label in zip(X,y):
            data[label].append(f)
        self.model = {label:self.summarize(value) for label, value in data.items()}
        return 'gaussianNB train done!'
    
    #计算概率
    def calculate_probabilities(self, input_data):
        probabilities = {}
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                mean, stdev = value[i]
                probabilities[label] *= self.gaussian_probability(input_data[i], mean, stdev)
        return probabilities
    
    #类别
    def predict(self, X_test):
        label = sorted(self.calculate_probabilities(X_test).items(),key = lambda x:x[-1])[-1][0]
        return label
    
    def score(self,X_test,y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1
                
        return right / float(len(X_test))

In [12]:
model = NaiveBayes()

In [13]:
model.fit(X_train,y_train)

'gaussianNB train done!'

In [14]:
print(model.predict([4.4, 3.2, 1.3, 0.2]))

0.0


In [15]:
model.score(X_test,y_test)

1.0

## scikit-learn实例##

In [16]:
from sklearn.naive_bayes import GaussianNB

In [17]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None)

In [19]:
clf.predict([[4.4, 3.2, 1.3, 0.2]])

array([0.])

In [20]:
clf.score(X_test, y_test)

1.0

In [21]:
from sklearn.naive_bayes import BernoulliNB

In [22]:
ber = BernoulliNB()
ber.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [23]:
ber.predict([[4.4,3.2,1.3,0.2]])

array([1.])

In [24]:
ber.score(X_test, y_test)

0.4

In [25]:
from sklearn.naive_bayes import MultinomialNB

In [26]:
Mul = MultinomialNB()
Mul.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
Mul.predict([[4.4, 3.2, 1.3, 0.2]])

array([0.])

In [28]:
Mul.score(X_test, y_test)

1.0